# 畳み込み層の実装
畳み込み層の具体的な実装方法を解説します。  
順伝播、逆伝播のコードを実装していきましょう。

## ●畳み込み層のクラス
畳み込み層は、以下のようにクラスとして実装します。  

In [ ]:
# -- 畳み込み層 --
class ConvLayer:
    
    # n_bt:バッチサイズ, x_ch:入力チャンネル数, x_h:入力画像高さ, x_w:入力画像幅
    # n_flt:フィルタ数, flt_h:フィルタ高さ, flt_w:フィルタ幅
    # stride:ストライド幅, pad:パディング幅
    # y_ch:出力チャンネル数, y_h:出力高さ, y_w:出力幅
    
    def __init__(self, x_ch, x_h, x_w, n_flt, flt_h, flt_w, stride, pad):

        # パラメータをまとめる
        self.params = (x_ch, x_h, x_w, n_flt, flt_h, flt_w, stride, pad)
        
        # フィルタとバイアスの初期値
        self.w = wb_width * np.random.randn(n_flt, x_ch, flt_h, flt_w)
        self.b = wb_width * np.random.randn(1, n_flt)
        
        # 出力画像のサイズ
        self.y_ch = n_flt  # 出力チャンネル数
        self.y_h = (x_h - flt_h + 2*pad) // stride + 1  # 出力高さ
        self.y_w = (x_w - flt_w + 2*pad) // stride + 1  # 出力幅

        ...

__init__メソッドでは、変更したり外部からアクセスしたりする必要のないパラメータを`self.params`にまとめています。  
また、フィルタ（`self.w`）とバイアス（`b`）の初期値を設定しています。  
フィルタはニューロンの重みと同じように扱うことができるため、変数名は`w`としています。

出力画像のチャンネル数、高さ、幅は、外部からアクセス可能にするために`self`が付けられています。  
このクラスに、メソッドとして順伝播と逆伝播を実装します。

## ●畳み込み層の順伝播
畳み込み層の順伝播を図で表すと、以下のようになります。

<img src="images/conv_forward.png">

この図を踏まえて、畳み込み層における順伝播を以下のようにメソッドとして実装します。

In [ ]:
    def forward(self, x):
        n_bt = x.shape[0] 
        x_ch, x_h, x_w, n_flt, flt_h, flt_w, stride, pad = self.params
        y_ch, y_h, y_w = self.y_ch, self.y_h, self.y_w
        
        # 入力画像とフィルタを行列に変換
        self.cols = im2col(x, flt_h, flt_w, y_h, y_w, stride, pad)
        self.w_col = self.w.reshape(n_flt, x_ch*flt_h*flt_w)
        
        # 出力の計算: 行列積、バイアスの加算、活性化関数
        u = np.dot(self.w_col, self.cols).T + self.b
        self.u = u.reshape(n_bt, y_h, y_w, y_ch).transpose(0, 3, 1, 2)
        self.y = np.where(self.u <= 0, 0, self.u)

`self`が付いている変数は、他のメソッドと共有するインスタンス変数です。  

メソッド内では、最初にim2colで入力画像を行列`self.w_col`に変換します。  
本セクションは、あらかじめ用意されたim2colのための関数を使用します。  
この関数には、入力画像、フィルタ高さ、幅、出力画像の高さ、幅、ストライドの幅、パディングの幅を渡します。  
そして、im2colにより変換された行列を返り値として得ることができます。  

また、フィルタはreshapeにより行列`self.w_col`に変換しておきます。  

`self.w_col`と`self.cols`の行列積をNumpyのdot関数により計算します。
この行列積の結果を、バイアスと列数を合わせるために転置した上で、バイアスを加えて`u`とします。 

そして、reshapeとtransposeにより形状を整えて`self.u`とします。  
transposeはNumPyの関数で、行列の軸を入れ替えることができます。  
これにより、バッチやチャンネル、画像の幅や高さなどの順番を配列内で入れ替えることができます。  
transposeについて詳しく知りたい方は、セルに簡単なコードを書いて配列の軸を入れ替えてみるとtransposeの特性が把握できるかと思います。  
これにより、`self.u`は入力画像と同じようにバッチとチャンネルに対応することになります。  

最後に、活性化関数でこの`self.u`を処理して、順伝播の出力とします。  
活性化関数にはReLUを使います。  

以上が順伝播の実装になりますが、フィルタを重みに置き換えれば、通常のニューラルネットワークの層における順伝播に似ているかと思います。

## ●畳み込み層の逆伝播
畳み込み層においては、以下の図のようにして各勾配を求めることができます。

<img src="images/conv_backward.png">

まず$\delta$を出力の勾配と活性化関数の微分をかけて求めます。    
そして、フィルタの勾配をcolsと$\delta$の行列積により求めます。  
バイアスの勾配は$\delta$そのままです。  

そして、colsの勾配をdeltaとフィルタの行列積で求めます。    
このcolsの勾配を、col2imにより入力の勾配に変換します。    

col2im以外は通常のニューラルネットワークにおける逆伝播と似ていますね。  
この図を踏まえて、逆伝播を以下のようにメソッドとして実装します。

In [ ]:
    def backward(self, grad_y):
        n_bt = grad_y.shape[0]
        x_ch, x_h, x_w, n_flt, flt_h, flt_w, stride, pad = self.params
        y_ch, y_h, y_w = self.y_ch, self.y_h, self.y_w
        
        # delta
        delta = grad_y * np.where(self.u <= 0, 0, 1)
        delta = delta.transpose(0,2,3,1).reshape(n_bt*y_h*y_w, y_ch)
        
        # フィルタとバイアスの勾配
        grad_w = np.dot(self.cols, delta)
        self.grad_w = grad_w.T.reshape(n_flt, x_ch, flt_h, flt_w)
        self.grad_b = np.sum(delta, axis=0)
        
        # 入力の勾配
        grad_cols = np.dot(delta, self.w_col)
        x_shape = (n_bt, x_ch, x_h, x_w)
        self.grad_x = col2im(grad_cols.T, x_shape, flt_h, flt_w, y_h, y_w, stride, pad)

このメソッドでは、出力の勾配（下の層の入力の勾配）を引数として受け取ります。  
これに活性化関数の微分形をかけて`delta`とします。  
今回は、ReLUの微分形をかけています。  
その上で、transposeとreshapeにより、`delta`を行列に変換します。  

次に、フィルタとバイアスの勾配を求めます。  
フィルタの勾配を求めるためには、まず`self.cols`と`delta`の行列積を求めます。   
この`grad_w`を、転置し、reshapeすることでフィルタの形状に合わせます。  
これにより、フィルタの勾配`self.grad_w`を求めることができます。  
バイアスの勾配は、`delta`の各列がフィルタに対応しているので、列ごとの総和により求めます。  

最後に、入力の勾配を求めます。  
そのためにまず、`delta`と順伝播の際に求めた`self.w_col`の行列積によりcolsの勾配を求めます。

この行列を転置したものは`self.cols`と同じ形状になります。  
これをcol2imで画像の形状に変換し入力の勾配`self.grad_x`とします。  
本セクションは、あらかじめ用意されたcol2imのための関数を使用します。  
この関数には、変換前の行列、入力画像の形状、フィルタ高さ、幅、出力画像の高さ、幅、ストライドの幅、パディングの幅を渡します。  
この入力の勾配`self.grad_x`が、上の層に伝播します。